In [ ]:
#HTML 템플릿 준비
html_template = """
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{user_id} - 멀티 페르소나 분석 결과</title>
    <style>
        * {{ margin: 0; padding: 0; box-sizing: border-box; }}
        body {{ font-family: 'Segoe UI', -apple-system, sans-serif; line-height: 1.6; color: #333; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; }}
        .container {{ max-width: 900px; margin: 0 auto; padding: 40px 20px; background: white; border-radius: 20px; box-shadow: 0 20px 40px rgba(0,0,0,0.1); }}
        .header {{ text-align: center; margin-bottom: 40px; }}
        .header h1 {{ color: #4a5568; font-size: 2.5em; margin-bottom: 10px; }}
        .ego-section {{ margin-bottom: 40px; padding: 30px; border-radius: 15px; }}
        .본능 자아 {{ background: linear-gradient(135deg, #ff9a9e 0%, #fecfef 100%); }}
        .관계 자아 {{ background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); }}
        .목표 자아 {{ background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%); }}
        .ego-title {{ font-size: 1.8em; margin-bottom: 20px; color: #2d3748; }}
        .percent-circle {{ width: 120px; height: 120px; border-radius: 50%; display: inline-flex; align-items: center; justify-content: center; font-size: 2em; font-weight: bold; margin: 20px 0; }}
        .본능 자아 .percent-circle {{ background: rgba(255,107,107,0.2); color: #FF6B6B; }}
        .관계 자아 .percent-circle {{ background: rgba(78,205,196,0.2); color: #4ECDC4; }}
        .목표 자아 .percent-circle {{ background: rgba(69,183,209,0.2); color: #45B7D1; }}
        .themes-list {{ display: flex; flex-wrap: wrap; gap: 10px; margin: 20px 0; }}
        .theme-tag {{ padding: 8px 16px; border-radius: 25px; font-weight: 500; font-size: 0.9em; }}
        .wordcloud {{ max-width: 100%; height: 250px; border-radius: 15px; margin: 20px 0; }}
        .summary {{ background: rgba(255,255,255,0.8); padding: 20px; border-radius: 10px; margin: 20px 0; font-style: italic; }}
        .guide {{ background: rgba(255,255,255,0.9); padding: 25px; border-left: 5px solid; margin: 20px 0; border-radius: 10px; }}
        .balance-guide {{ background: linear-gradient(135deg, #a8e6cf 0%, #dcedc1 100%); border-left-color: #56ab2f; }}
        @media (max-width: 768px) {{ .container {{ padding: 20px 15px; }} .ego-section {{ padding: 20px; }} }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>👤 {user_id}</h1>
            <p>멀티 페르소나 분석 결과</p>
        </div>

        {ego_sections}

        <div class="guide balance-guide">
            <h3>🌈 자아 균형 가이드</h3>
            <div class="summary">{balance_guide}</div>
        </div>
    </div>
</body>
</html>
"""

print("✅ HTML 템플릿 준비 완료!")


In [ ]:
#사용자별 HTML 생성 함수
def create_user_html(user_id):
    """한 사용자 HTML 결과지 생성"""
    # 사용자 데이터 로드
    with open(f"mp_survey/users/{user_id}_report.json", "r", encoding="utf-8") as f:
        user_report = json.load(f)

    ego_sections = ""
    for ego in ['본능 자아', '관계 자아', '목표 자아']:
        data = user_report['ego_overview'][ego]
        percent = data['percent']
        top_themes = data['top_themes']
        # summary = user_report['llm_summary']['summary'][ego]  # LLM 결과

        themes_html = ""
        for theme, count in top_themes:
            themes_html += f'<span class="theme-tag">{theme} ({count})</span>'

        ego_sections += f"""
        <div class="ego-section {ego}">
            <div class="ego-title">{ego}</div>
            <div class="percent-circle">{percent}%</div>
            <div class="themes-list">{themes_html}</div>
            <img src="wordclouds/{user_id}_wordclouds.png" class="wordcloud" alt="{ego} 워드클라우드">
            <div class="summary">
                {ego} 탐색 정도가 {percent}%입니다.
                주요 키워드: {', '.join([t for t,c in top_themes[:3]])}
            </div>
        </div>
        """

    balance_guide = user_report['llm_summary']['balance_guide']

    # HTML 완성
    html_content = html_template.format(
        user_id=user_id,
        ego_sections=ego_sections,
        balance_guide=balance_guide
    )

    # 저장 (랜덤 토큰 URL용)
    token = f"{user_id}_{np.random.randint(1000,9999):04d}"
    html_path = f"mp_survey/users/{token}.html"

    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

    print(f"✅ {user_id} → {token}.html 생성 완료!")
    return token, html_path

print("✅ HTML 생성 함수 준비 완료!")


In [ ]:
# 🎉 모든 사용자 결과 HTML 생성
html_index = """
<!DOCTYPE html>
<html>
<head><title>멀티 페르소나 결과</title></head>
<body>
<h1>👥 멀티 페르소나 분석 결과</h1>
<ul>
"""

user_htmls = []
for user_id in list(all_user_analysis.keys()):
    token, html_path = create_user_html(user_id)
    user_htmls.append((user_id, token))

    # 인덱스 페이지에 링크 추가
    html_index += f'<li><a href="{token}.html">{user_id}</a></li>\n'

html_index += """
</ul>
<p><em>개별 링크로 공유하세요!</em></p>
</body>
</html>
"""

# 인덱스 페이지 저장
with open("mp_survey/users/index.html", "w", encoding="utf-8") as f:
    f.write(html_index)

print("\n🎊 모든 HTML 생성 완료!")
print(f"📁 mp_survey/users/ 폴더 확인:")
print(os.listdir("mp_survey/users/"))


In [ ]:
#배포준비
# GitHub Pages용 압축
import shutil
import zipfile

# users/ 폴더 전체 압축
shutil.make_archive("mp_survey_results", "zip", "mp_survey/users")

# 다운로드
from google.colab import files
files.download("mp_survey_results.zip")

print("""
🚀 배포 방법:

1. 압축파일 다운로드 → 압축해제
2. GitHub Pages: users/ 폴더를 gh-pages 브랜치에 올리기
3. Netlify: users/ 폴더 드래그앤드롭
4. Vercel: users/ 폴더 임포트

📋 공유 링크 예시:
- 전체 인덱스: https://your-site.com/index.html
- 사용자 A: https://your-site.com/u001_1234.html
""")
